In [88]:
import tensorflow as tf
#import libraries 
import os
import cv2
import matplotlib.pyplot as plt
import re
from sklearn.cluster import DBSCAN  # For DBSCAN
import numpy as np
%matplotlib inline


In [117]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D, Convolution2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import Model
from keras.preprocessing.image import ImageDataGenerator
#from tf.keras.optimizers import SGD
from sklearn.metrics import confusion_matrix

In [61]:

image_paths=[] #List to store path of all images

for dirname, _, filenames in os.walk('../Dataset/MICC-F220'):
    for filename in filenames:
        if '.txt' in filename:
            continue
        image_paths.append(os.path.join(dirname, filename))
        
        
# prepare dataset
original_images=[]
tampered_images=[]
x = []
y = []
for path in image_paths:
    
    if 'tamp' in path:              # As Observed from the above list tampered images name has tamp
        img = cv2.imread(path)
        img_resize = cv2.resize(img,(224,224))
        img_scale = img_resize/255.0
        x.append(img_scale)
        y.append(0)

        
        tampered_images.append(path)
    else:
        img = cv2.imread(path)
        img_resize = cv2.resize(img,(224,224))
        img_scale = img_resize/255.0
        x.append(img_scale)
        y.append(1)
        original_images.append(path)
tampered_images.sort()
original_images.sort()
print(len(original_images),len(tampered_images))


110 110


In [62]:
X = np.array(x)
Y = np.array(y)

In [63]:
X.shape, Y.shape
Y[111]

1

# model building

In [ ]:
# Download the architecture of ResNet50 with ImageNet weights
base_model = ResNet50(include_top=False, weights='imagenet')

In [ ]:
conv_base = ResNet50(include_top=False,input_shape=(240,240,3), pooling='avg', weights='imagenet')


for layer in conv_base.layers:
    layer.trainable = False


In [74]:

# Taking the output of the last convolution block in ResNet50
x = base_model.output
 
# Adding a Global Average Pooling layer
x = GlobalAveragePooling2D()(x)
 
# Adding a fully connected layer having 1024 neurons
x = Dense(512, activation='relu')(x)
x = Dense(240, activation='relu')(x)

# Adding a fully connected layer having 2 neurons which will
# give the probability of image having either dog or cat
predictions = Dense(2, activation='softmax')(x)
# Model to be trained
model = Model(inputs=base_model.input, outputs=predictions)
 
# Training only top layers i.e. the layers which we have added in the end
for layer in base_model.layers:
    layer.trainable = False

#Compiling the model
#model.compile(optimizer= tf.keras.optimizers.SGD(lr=0.0001, momentum=0.9), loss='sparse_categorical_crossentropy', metrics = ['accuracy'])
model.compile(loss='sparse_categorical_crossentropy', optimizer="rmsprop", metrics=['accuracy']) 

In [75]:
history = model.fit(X,Y,steps_per_epoch=100 // 32,batch_size =12,  epochs=30)
#model.fit(train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)


Epoch 1/30
3/3 [==============================] - 5s 862ms/step - loss: 1.1944 - accuracy: 0.5278
Epoch 2/30
3/3 [==============================] - 3s 851ms/step - loss: 1.0454 - accuracy: 0.6111
Epoch 3/30
3/3 [==============================] - 3s 845ms/step - loss: 0.7884 - accuracy: 0.6111
Epoch 4/30
3/3 [==============================] - 3s 842ms/step - loss: 0.9450 - accuracy: 0.4167
Epoch 5/30
3/3 [==============================] - 3s 840ms/step - loss: 0.6788 - accuracy: 0.5000
Epoch 6/30
3/3 [==============================] - 3s 838ms/step - loss: 0.7157 - accuracy: 0.4722
Epoch 7/30
3/3 [==============================] - 2s 844ms/step - loss: 0.6880 - accuracy: 0.5000
Epoch 8/30
3/3 [==============================] - 3s 871ms/step - loss: 0.7132 - accuracy: 0.3333
Epoch 9/30
3/3 [==============================] - 3s 844ms/step - loss: 0.7186 - accuracy: 0.3333
Epoch 10/30
3/3 [==============================] - 3s 841ms/step - loss: 0.6984 - accuracy: 0.5556
Epoch 11/30
3/3 [==

KeyboardInterrupt: 

In [94]:
Classifier =Sequential()

In [95]:
Classifier.add(Convolution2D(32,3,3,input_shape=(224,224,3),activation='relu'))
Classifier.add(MaxPooling2D(pool_size=(2,2)))
Classifier.add(Convolution2D(128,3,3,activation='relu'))
Classifier.add(MaxPooling2D(pool_size=(2,2)))
Classifier.add(Flatten())
Classifier.add(Dense(256, activation='relu'))
Classifier.add(Dense(2, activation="softmax"))

Classifier.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
Classifier.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_22 (Conv2D)          (None, 74, 74, 32)        896       
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 37, 37, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_23 (Conv2D)          (None, 12, 12, 128)       36992     
                                                                 
 max_pooling2d_19 (MaxPoolin  (None, 6, 6, 128)        0         
 g2D)                                                            
                                                                 
 flatten_9 (Flatten)         (None, 4608)              0         
                                                                 
 dense_39 (Dense)            (None, 256)              

In [97]:
#Classifier.fit( X,Y, steps_per_epoch=100/12, batch_size = 12, epochs=12)
history = Classifier.fit(X,Y,steps_per_epoch=100 // 32,batch_size =12,  epochs=30)
#model.fit(train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)


Epoch 1/30
3/3 [==============================] - 0s 34ms/step - loss: 0.2197 - accuracy: 0.8889
Epoch 2/30
3/3 [==============================] - 0s 34ms/step - loss: 0.3697 - accuracy: 0.9167
Epoch 3/30
3/3 [==============================] - 0s 34ms/step - loss: 0.2493 - accuracy: 0.9722
Epoch 4/30
3/3 [==============================] - 0s 34ms/step - loss: 0.3415 - accuracy: 0.8889
Epoch 5/30
3/3 [==============================] - 0s 34ms/step - loss: 0.2656 - accuracy: 0.8889
Epoch 6/30
3/3 [==============================] - 0s 35ms/step - loss: 0.1826 - accuracy: 0.9722
Epoch 7/30
3/3 [==============================] - 0s 38ms/step - loss: 0.2574 - accuracy: 0.9286
Epoch 8/30
3/3 [==============================] - 0s 35ms/step - loss: 0.1818 - accuracy: 0.9722
Epoch 9/30
3/3 [==============================] - 0s 36ms/step - loss: 0.2861 - accuracy: 0.9167
Epoch 10/30
3/3 [==============================] - 0s 35ms/step - loss: 0.3319 - accuracy: 0.8056
Epoch 11/30
3/3 [============

In [98]:
X[1].shape

(224, 224, 3)

In [107]:
X

array([[[[0.58039216, 0.61960784, 0.65882353],
         [0.6       , 0.63921569, 0.67843137],
         [0.58039216, 0.61960784, 0.65882353],
         ...,
         [0.78431373, 0.78431373, 0.78431373],
         [0.8627451 , 0.8627451 , 0.8627451 ],
         [0.86666667, 0.86666667, 0.86666667]],

        [[0.59215686, 0.63137255, 0.67058824],
         [0.58431373, 0.62352941, 0.6627451 ],
         [0.59215686, 0.63137255, 0.67058824],
         ...,
         [0.72156863, 0.72156863, 0.72156863],
         [0.85490196, 0.85490196, 0.85490196],
         [0.87058824, 0.87058824, 0.87058824]],

        [[0.61960784, 0.65882353, 0.69803922],
         [0.56862745, 0.60784314, 0.64705882],
         [0.57647059, 0.61568627, 0.65490196],
         ...,
         [0.83921569, 0.83921569, 0.83921569],
         [0.85882353, 0.85882353, 0.85882353],
         [0.8627451 , 0.8627451 , 0.8627451 ]],

        ...,

        [[0.88235294, 0.83529412, 0.81176471],
         [0.86666667, 0.81960784, 0.79607843]

In [108]:
res = Classifier.predict((X))
result = np.argmax(res, axis = 1)

In [114]:
Y[40:60]

array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [115]:
result[40:60]

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
      dtype=int64)

In [121]:
#compute the confusion matrix
confusion_mtx = confusion_matrix(Y, result) 
confusion_mtx

array([[110,   0],
       [ 11,  99]], dtype=int64)